In [1]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
from skyModel import skyModel

In [2]:
class skyDataset(Dataset):
    def __init__(self, dataset_dim, data_path, label_path):
        super().__init__()
        self.image_w = 125
        self.data = np.memmap(data_path, dtype='float32', mode='r', shape=(dataset_dim, 3, self.image_w, self.image_w))
        self.label = np.memmap(label_path, dtype='uint8', mode='r', shape=(dataset_dim,))
        self.size = dataset_dim

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        image = self.data[idx]
        image = torch.from_numpy(np.array(image))

        label = torch.tensor(self.label[idx], dtype=torch.float32)

        return image, label

In [5]:
def eval_acc(model, data_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y_true in data_loader:

            y_pred = model(x)
            y_pred = torch.squeeze(y_pred)
            y_pred = torch.round(y_pred)

            total += y_true.size(0)
            c = (y_pred == y_true).sum().item()
            correct += c

    return correct / total

In [3]:
train_data = os.path.join(".","dataset", "train_data.npm")
train_label = os.path.join(".","dataset", "train_label.npm")
test_data = os.path.join(".","dataset", "test_data.npm")
test_label = os.path.join(".","dataset", "test_label.npm")

train_dataset = skyDataset(7200, train_data, train_label)
test_dataset = skyDataset(2178, test_data, test_label)

dl_train = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
dl_test = DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=0)

In [4]:
learning_rate = 0.01
epoch_num = 10

model = skyModel()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_values = []
train_accs = []
test_accs = []

for i in range(epoch_num):
    for j, (x, y_true) in enumerate(dl_train):
        y_pred = model(x)
        y_pred = torch.squeeze(y_pred)

        loss = loss_fn(y_pred, y_true)
        l = loss.item()
        loss_values.append(l)
        #print("Epoch:", i, "Batch:", j, "Loss:", l)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if j%100 == 0:
            train_acc = eval_acc(model, dl_train)
            test_acc = eval_acc(model, dl_test)
            print("Epoch:", i, "Batch:", j)
            print("Train acc:", train_acc)
            print("Test acc:", test_acc , "\n")
            train_accs.append(train_acc)
            test_accs.append(test_accs)


torch.save(model.state_dict(), "skyModel.bin")
#print(loss_values)
#print(train_accs)
#print(test_accs)

print("final train accuracy: ", eval_acc(model, dl_train))
print("final test accuracy: ", eval_acc(model, dl_test))

NameError: name 'eval_acc' is not defined